# Imports

In [1]:
import json
import re
import unicodedata

import nltk
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

import flair

import wikipedia

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jkutc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jkutc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jkutc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
names = ['jacob', 'winston', 'niyi']

In [3]:
with open('../Twitter_Scraper/Data/' + names[0] + '_tweets.json') as json_data:
    jacob_tweets_json_from_file = json.load(json_data)

In [4]:
with open('../Twitter_Scraper/Data/' + names[1] + '_tweets.json') as json_data:
    winston_tweets_json_from_file = json.load(json_data)

In [5]:
with open('../Twitter_Scraper/Data/' + names[2] + '_tweets.json') as json_data:
    niyi_tweets_json_from_file = json.load(json_data)

In [6]:
niyi_tweets_json_from_file[0]

{'created_at': 'Sat Jan 14 23:59:52 +0000 2017',
 'id': 820420199774035970,
 'id_str': '820420199774035970',
 'text': 'RT @nytimes: SpaceX launches rocket, its first since a different Falcon 9 caught fire and exploded in Florida  https://t.co/5GdkDkC1BX',
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 735885465006575617,
  'id_str': '735885465006575617',
  'name': 'News RT World🌍',
  'screen_name': 'NewsRTWorld',
  'location': 'Brasília, Brasil',
  'url': None,
  'description': 'RT do Brasil e do Mundo.\n\n\nRT Brasil y del mundo.\n\n\nRT Brésil et du monde.\n\n\nRT Brazil and the world.\n       \n@NewsRTWorld',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 811,
  'friends_count': 755,
  '

In [7]:
niyi_tweets_json_from_file[-1]

{'created_at': 'Mon Jan 30 23:59:15 +0000 2017',
 'id': 826218248731365376,
 'id_str': '826218248731365376',
 'text': 'Justice department lawyers instructed not to defend Trump migration order – reports https://t.co/xhjG6Glyhu https://t.co/TmUZRy3NxR',
 'display_text_range': [0, 107],
 'source': '<a href="https://dlvrit.com/" rel="nofollow">dlvr.it</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 48470839,
  'id_str': '48470839',
  'name': 'Patrick Willey',
  'screen_name': 'cahulaan',
  'location': 'Arizona, USA',
  'url': None,
  'description': "🌵🌊 #TheResistance #BLM #MeToo #StayingBlueIn22  🚫 No DM's #HoldThemAllAccountable ✊🏻'E Pluribus Unum' 🗽",
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 18344,
  'friends_count': 18368,
  'listed_count': 22,
  'favourites_count': 68504,
  '

In [8]:
len(niyi_tweets_json_from_file)

400

In [9]:
tweet_list = jacob_tweets_json_from_file + winston_tweets_json_from_file + niyi_tweets_json_from_file

# Preprocessing Functions

In [10]:
def set_geo(tweet):
    if tweet['geo'] != None:
        return tweet['geo']
    elif tweet['user']['location'] != None:
        return tweet['user']['location']
    else:
        return None 

In [11]:
def remove_urls_from_text(text):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", text)

In [12]:
def remove_non_ascii(text):
    """Remove non-ASCII characters from list of tokenized words"""
    new_text = []
    for word in text:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_text.append(new_word)
    return new_text

In [13]:
def filter_tweet_text(text):
    # Remove Urls
    text = remove_urls_from_text(text)
    
    #Tokenization
    nltk_tokenList = word_tokenize(text)
    
    # Remove non ascii (Emojis,etc)
    nltk_tokenList = remove_non_ascii(nltk_tokenList)
    
    #Stemming
    p_stemmer = PorterStemmer()
    nltk_stemedList = []
    for word in nltk_tokenList:
        nltk_stemedList.append(p_stemmer.stem(word))
    
    #Lemmatization
    wordnet_lemmatizer = WordNetLemmatizer()
    nltk_lemmaList = []
    for word in nltk_stemedList:
        nltk_lemmaList.append(wordnet_lemmatizer.lemmatize(word))  
    
    #Filter stopwords
    filtered_sentence = []  
    nltk_stop_words = set(stopwords.words("english"))
    for w in nltk_lemmaList:  
        if w not in nltk_stop_words:  
            filtered_sentence.append(w)    #Removing Punctuation
    
    # Remove punctuation
    punctuations="?:!.,;"
    for word in filtered_sentence:
        if word in punctuations:
            filtered_sentence.remove(word)

    return filtered_sentence

In [14]:
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

2021-11-11 17:13:22,281 loading file C:\Users\jkutc\.flair\models\sentiment-en-mix-distillbert_4.pt


In [15]:
def sentiment_analysis(text):
    # Remove Urls
    text = remove_urls_from_text(text)
    
    sentence = flair.data.Sentence(text)
    flair_sentiment.predict(sentence)
    total_sentiment = sentence.labels[0]
    assert total_sentiment.value in ['POSITIVE', 'NEGATIVE']
    sign = 1 if total_sentiment.value == 'POSITIVE' else -1
    score = total_sentiment.score
    
    if sign * score > 0.1:
        return 'POSITIVE'
    elif sign * score < -0.1:
        return 'NEGATIVE'
    else:
        return 'NEUTRAL'

In [16]:
flair_ner = flair.models.SequenceTagger.load('ner')

2021-11-11 17:13:23,299 --------------------------------------------------------------------------------
2021-11-11 17:13:23,299 The model key 'ner' now maps to 'https://huggingface.co/flair/ner-english' on the HuggingFace ModelHub
2021-11-11 17:13:23,300  - The most current version of the model is automatically downloaded from there.
2021-11-11 17:13:23,300  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/ner/en-ner-conll03-v0.4.pt)
2021-11-11 17:13:23,300 --------------------------------------------------------------------------------
2021-11-11 17:13:23,348 loading file C:\Users\jkutc\.flair\models\ner-english\4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4


In [17]:
def named_entity_recognition(text):
    text = remove_urls_from_text(text)
    sentence = flair.data.Sentence(text)
    flair_ner.predict(sentence)
    entities_list = sentence.to_dict(tag_type='ner')['entities']
    return entities_list

In [18]:
def generate_wikipedia_links(named_entities):
    for i in range(len(named_entities)):
        del named_entities[i]['labels']
        try:
             named_entities[i]['wiki_url'] = wikipedia.page(named_entities[i]['text']).url      
        except:
            continue

# Preprocessing

In [19]:
preprocessed_tweet_list = []

In [20]:
for i in range(len(tweet_list)):
    temp_user = {
        'id' : tweet_list[i]['user']['id'],
        'name' : tweet_list[i]['user']['name'],
        'screen_name' : tweet_list[i]['user']['screen_name'],
        'location' : tweet_list[i]['user']['location'],
        'url' : tweet_list[i]['user']['url'],
        'description' : tweet_list[i]['user']['description'],
        'verified' : tweet_list[i]['user']['verified'],
        'followers_count' : tweet_list[i]['user']['followers_count'],
        'statuses_count' : tweet_list[i]['user']['statuses_count'],
        'created_at' : tweet_list[i]['user']['created_at'],
        'geo_enabled' : tweet_list[i]['user']['geo_enabled'],
        'lang' : tweet_list[i]['user']['lang'],
    }

    named_entities_list = named_entity_recognition(tweet_list[i]['text'])
    generate_wikipedia_links(named_entities_list)

    temp_tweet = {
        'created_at' : tweet_list[i]['created_at'],
        'id' : tweet_list[i]['id'],
        'text' : tweet_list[i]['text'],
        'geo' : set_geo(tweet_list[i]),
        'coordinates' : tweet_list[i]['coordinates'],
        'quote_count' : tweet_list[i]['quote_count'],
        'reply_count': tweet_list[i]['reply_count'],
        'retweet_count': tweet_list[i]['retweet_count'],
        'favorite_count': tweet_list[i]['favorite_count'],
        'entities' : tweet_list[i]['entities'],
        'lang' : tweet_list[i]['lang'],
        'sentiment' : sentiment_analysis(tweet_list[i]['text']),
        'named_entities' :  named_entities_list,
        'user' : temp_user,
    }

    preprocessed_tweet_list.append(temp_tweet)

C:\Users\jkutc\anaconda3\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\jkutc\anaconda3\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [37]:
preprocessed_tweet_list[0]

{'created_at': 'Mon Sep 04 19:07:25 +0000 2006',
 'id': 25471,
 'text': "Steve Irwin* is dead! Betrayed by a stingray's 'dirty dagger.'",
 'geo': 'Gilbert, AZ',
 'coordinates': None,
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 3,
 'favorite_count': 0,
 'entities': {'hashtags': [], 'urls': [], 'user_mentions': [], 'symbols': []},
 'lang': 'en',
 'sentiment': 'NEGATIVE',
 'named_entities': [{'text': 'Steve Irwin',
   'start_pos': 0,
   'end_pos': 11,
   'wiki_url': 'https://en.wikipedia.org/wiki/Steve_Irwin'}],
 'user': {'id': 4465,
  'name': 'Adam',
  'screen_name': 'adammachado',
  'location': 'Gilbert, AZ',
  'url': None,
  'description': 'CA, OR, AZ',
  'verified': False,
  'followers_count': 84,
  'statuses_count': 1403,
  'created_at': 'Fri Aug 18 01:20:47 +0000 2006',
  'geo_enabled': True,
  'lang': None}}

# Save Preprocessed Tweets

In [23]:
def writeToJSONFile(path, fileName, data):
    try:
        filePathNameWExt = path + fileName
        with open(filePathNameWExt, 'w') as fp:
            json.dump(data, fp)
    except Exception as e:
        if(not quiet):
            print("writeToJSONFile exception")
            print(e)

## List Form

In [39]:
json_preprocessed_tweet_list = json.loads(json.dumps(preprocessed_tweet_list))

In [ ]:
writeToJSONFile('Preprocessed_Data/', 'preprocessed_tweets_list.json', json_preprocessed_tweet_list)

In [ ]:
with open('Preprocessed_Data/preprocessed_tweets_list.json') as json_data:
    tweets_json_from_file = json.load(json_data)

Verify Tweets list and saved json file contain the same Tweets

In [29]:
len(preprocessed_tweet_list) == len(tweets_json_from_file)

True

In [30]:
preprocessed_tweet_list == tweets_json_from_file

True

## Dictionary Form

In [32]:
preprocessed_tweet_dict = {}

In [33]:
for i in range(len(preprocessed_tweet_list)):
    if preprocessed_tweet_list[i]['id'] not in preprocessed_tweet_dict:
        preprocessed_tweet_dict[preprocessed_tweet_list[i]['id']] = preprocessed_tweet_list[i]
    else: 
        print('Repeated ID')
        stop()
    

In [36]:
len(preprocessed_tweet_dict) == len(preprocessed_tweet_list)

True

In [38]:
preprocessed_tweet_dict[25471]

{'created_at': 'Mon Sep 04 19:07:25 +0000 2006',
 'id': 25471,
 'text': "Steve Irwin* is dead! Betrayed by a stingray's 'dirty dagger.'",
 'geo': 'Gilbert, AZ',
 'coordinates': None,
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 3,
 'favorite_count': 0,
 'entities': {'hashtags': [], 'urls': [], 'user_mentions': [], 'symbols': []},
 'lang': 'en',
 'sentiment': 'NEGATIVE',
 'named_entities': [{'text': 'Steve Irwin',
   'start_pos': 0,
   'end_pos': 11,
   'wiki_url': 'https://en.wikipedia.org/wiki/Steve_Irwin'}],
 'user': {'id': 4465,
  'name': 'Adam',
  'screen_name': 'adammachado',
  'location': 'Gilbert, AZ',
  'url': None,
  'description': 'CA, OR, AZ',
  'verified': False,
  'followers_count': 84,
  'statuses_count': 1403,
  'created_at': 'Fri Aug 18 01:20:47 +0000 2006',
  'geo_enabled': True,
  'lang': None}}

In [24]:
json_preprocessed_tweet_dict = json.loads(json.dumps(preprocessed_tweet_dict))

In [25]:
writeToJSONFile('Preprocessed_Data/', 'preprocessed_tweets_dict.json', json_preprocessed_tweet_dict)

In [26]:
with open('Preprocessed_Data/preprocessed_tweets_dict.json') as json_data:
    tweets_json_from_file = json.load(json_data)

Verify Tweets list and saved json file contain the same Tweets

In [29]:
len(preprocessed_tweet_dict) == len(json_preprocessed_tweet_dict)

True

In [30]:
preprocessed_tweet_dict == json_preprocessed_tweet_dict

True